In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from  tensorflow.keras.preprocessing.sequence import pad_sequences

2023-11-19 15:18:04.009235: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
restaurant_data = pd.read_csv("final_df.csv")

In [3]:
restaurant_data.columns

Index(['name', 'cuisine', 'lat', 'lon', 'rating', 'Has Menu', 'locale',
       'review_objects', 'dishes', 'price', 'type', 'popular_times',
       'in_cuisine', 'japanese', 'chinese', 'bar', 'western', 'indian',
       'mediterranean', 'mexican', 'italian', 'local', 'malaysian', 'hipster',
       'veg', 'fast_food', 'thai', 'korean', 'halal', 'dessert', 'seafood',
       'health', 'french', 'vietnamese', 'michellin'],
      dtype='object')

In [4]:
restaurant_data_clean  = restaurant_data.drop(['cuisine', 'lat', 'lon', 'rating', 'Has Menu', 'locale', 'review_objects', 'dishes', 'type', 'popular_times', 'in_cuisine'], axis = 1)

In [5]:
restaurant_data_clean['price'].unique()

array(['$$', '$', '$$$', '$$ - $$$', '$$$$', '$$$$$'], dtype=object)

In [6]:
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$', 'price'] = 0
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$', 'price'] = 1
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$ - $$$', 'price'] = 2
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$', 'price'] = 3
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$$', 'price'] = 4
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$$$', 'price'] = 5

In [7]:
restaurant_name_list = restaurant_data_clean['name']
restaurant_data_clean.drop(['name'], axis = 1, inplace = True)

In [8]:
restaurant_data_clean
## Create df from streamlit

,price,japanese,chinese,bar,western,indian,mediterranean,mexican,italian,local,...,fast_food,thai,korean,halal,dessert,seafood,health,french,vietnamese,michellin
0,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
2,3,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4199,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4200,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4201,3,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4202,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
restaurant_data_clean.to_csv('restaurant_data_clean.csv', index = False)

In [63]:
restaurant_numpy = restaurant_data_clean.drop('price', axis = 1).to_numpy()

In [64]:
# Save restaurant_numpy
np.save('restaurant_numpy.npy', restaurant_numpy)

In [65]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [66]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [67]:
print(restaurant_numpy, restaurant_numpy.shape)    

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]] (4204, 22)


In [68]:
restaurant_data_clean['price'].unique()

array([1, 0, 3, 2, 4, 5], dtype=object)

In [69]:
# Implement Content Based Filtering 
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
'''
# Price, Cuisine Not needed anymore
item = np.array([[1, 0, 1],[1, 1, 1], [0, 1, 1], [0, 0, 1]])
user = np.array([[0, 1, 1], [1, 1, 0]])
'''

'\n# Price, Cuisine Not needed anymore\nitem = np.array([[1, 0, 1],[1, 1, 1], [0, 1, 1], [0, 0, 1]])\nuser = np.array([[0, 1, 1], [1, 1, 0]])\n'

In [70]:
# Item
item = restaurant_numpy
# User
rng = np.random.default_rng(12345)
user_cusine = rng.integers(low = 0, high = 2, size = (3,22))
#price_user = rng.integers(low= 0, high = 6, size=(3,1))
#user = np.concatenate((price_user, user_cusine), axis = 1)
user = user_cusine
print(user, user.shape)
#print(np.random.rand(3,23))


[[1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0]
 [1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0]
 [1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1]] (3, 22)


In [71]:
# Save user array
np.save('user.npy', user)

In [72]:
def get_item_user(user_arr, item):
    user_item = []
    for user in user_arr:
        cos_sim = dot(user, item.T)/(norm(user)*norm(item))
        user_item.append(cos_sim)

    return user_item

In [73]:
item_user = get_item_user(user, item)
print(len(item_user), len(item_user[0]))
print(["{0:0.2f}".format(i) for i in item_user[0]])

3 4204
['0.01', '0.01', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.01', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.01', '0.00', '0.01', '0.00', '0.01', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.00', '0.01', '0.01', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.01', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', 

In [74]:
from sklearn.preprocessing import  MinMaxScaler

scaler_MinMax = MinMaxScaler()
def scale_item_user(item_user):
    for i in range(0, len(item_user)):
        reshaped_item = item_user[i].reshape(-1,1)
        scaled_item = scaler_MinMax.fit_transform(reshaped_item).reshape(1,-1)[0]
        item_user[i] = scaled_item

    return item_user

item_user = scale_item_user(item_user)


In [75]:
print(item_user[0])

[0.5  0.5  0.25 ... 0.25 0.25 0.25]


In [76]:
print(len(item_user[0]))

4204


In [77]:
index_of_item_user = sorted(range(len(item_user[0])), key=lambda i: item_user[0][i])[-3:]

In [78]:
index_of_item_user

[4142, 4150, 878]

In [79]:
for index in index_of_item_user:
    print(item_user[0][index])

0.75
0.75
1.0


In [80]:
for i in range(0, 23):
    print(restaurant_data_clean.columns[i], user[0][i])

price 1
japanese 0
chinese 1
bar 0
western 0
indian 1
mediterranean 1
mexican 1
italian 1
local 0
malaysian 1
hipster 0
veg 1
fast_food 1
thai 0
korean 0
halal 0
dessert 1
seafood 1
health 1
french 1
vietnamese 0


IndexError: index 22 is out of bounds for axis 0 with size 22

In [81]:
print(restaurant_data_clean.loc[1684,:])
print(restaurant_data_clean.loc[4176,:])
print(restaurant_data_clean.loc[4044,:])

price            5
japanese         0
chinese          0
bar              0
western          0
indian           0
mediterranean    0
mexican          0
italian          0
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             1
korean           0
halal            0
dessert          0
seafood          1
health           0
french           0
vietnamese       0
michellin        0
Name: 1684, dtype: object
price            5
japanese         0
chinese          0
bar              0
western          1
indian           0
mediterranean    0
mexican          0
italian          0
local            0
malaysian        0
hipster          1
veg              1
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 4176, dtype: object
price            5
japanese         1
chinese          0
bar              

In [83]:
def item_item(item):
    item_item = []
    for i in range(0, len(item)):
        item_similarity = []
        for j in range(0, len(item)):
            #if(i!=j):
            item_i = item[i]
            item_j = item[j]
            cos_sim = dot(item_i, item_j)/(norm(item_i)*norm(item_j))
            item_similarity.append(cos_sim)
        item_item.append(item_similarity)

    return item_item

item_item_matrix = item_item(item)
    
# Send as csv

In [84]:
# Save item_item_matrix
np.save('item_item_matrix.npy', item_item_matrix)

In [85]:
print(sorted(item_item_matrix[0], reverse = True))

[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475, 0.7071067811865475,

In [86]:
print(item[514])
print(item[0])
print(item_item_matrix[0][514])

[1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8164965809277259


In [87]:
index_of_item_item = sorted(range(len(item_item_matrix)), key=lambda i: item_item_matrix[i])[-4:]

In [88]:
for index in index_of_item_item:
    print(index)
    print(item_item_matrix[0][index])
#print(item_item_matrix[index_of_item_item])

1817
0.9999999999999998
2387
0.9999999999999998
2724
0.9999999999999998
3028
0.9999999999999998


In [89]:
print(restaurant_name_list[index_of_item_item] , restaurant_data_clean.loc[index_of_item_item, :])

1817                   kabe no ana
2387    momoya japanese restaurant
2724                    pastaholic
3028                       sg taps
Name: name, dtype: object      price  japanese  chinese  bar  western  indian  mediterranean  mexican  \
1817     2         1        0    0        0       0              0        0   
2387     2         1        0    0        0       0              0        0   
2724     2         1        0    0        0       0              0        0   
3028     2         1        0    0        0       0              0        0   

      italian  local  ...  fast_food  thai  korean  halal  dessert  seafood  \
1817        1      0  ...          0     0       0      0        0        0   
2387        1      0  ...          0     0       0      0        0        0   
2724        1      0  ...          0     0       0      0        0        0   
3028        1      0  ...          0     0       0      0        0        0   

      health  french  vietnamese  michelli

In [90]:
print(restaurant_data_clean.loc[2724, :])
print(restaurant_data_clean.loc[3028, :])
print(restaurant_data_clean.loc[0, :])

price            2
japanese         1
chinese          0
bar              0
western          0
indian           0
mediterranean    0
mexican          0
italian          1
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 2724, dtype: object
price            2
japanese         1
chinese          0
bar              0
western          0
indian           0
mediterranean    0
mexican          0
italian          1
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 3028, dtype: object
price            1
japanese         1
chinese          0
bar              

In [91]:
restaurant_data_clean[(restaurant_data_clean['price'] == 1) & (restaurant_data_clean['japanese'] == 1) & restaurant_data_clean['italian'] == 1]
japanese	chinese	bar	western	indian	mediterranean	mexican	italian	local	malaysian	hipster	veg	fast_food	thai	korean	halal	dessert	seafood	health	french	vietnamese	michellin

,price,japanese,chinese,bar,western,indian,mediterranean,mexican,italian,local,...,fast_food,thai,korean,halal,dessert,seafood,health,french,vietnamese,michellin
0,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
514,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1540,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2385,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2465,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2747,1,1,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [92]:
print(restaurant_name_list[514])
print(restaurant_data_clean.loc[514, :])

bold bowls
price            1
japanese         1
chinese          0
bar              0
western          1
indian           0
mediterranean    0
mexican          0
italian          1
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 514, dtype: object


In [93]:
print(item_item_matrix[1684])

print(np.shape(item_item_matrix))

[0.0, 0.35355339059327373, 0.0, 0.0, 0.0, 0.0, 0.4999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4999999999999999, 0.0, 0.0, 0.4999999999999999, 0.0, 0.0, 0.0, 0.0, 0.40824829046386296, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, 0.4999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.40824829046386296, 0.0, 0.4999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.40824829046386296, 0.0, 0.0, 0.0, 0.0, 0.40824829046386296, 0.0, 0.0, 0.4999999999999999, 0.0, 0.40824829046386296, 0.7071067811865475, 0.0, 0.0, 0.4999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.40824829046386296, 0.0, 0.0, 0.0, 0.4999999999999999, 0.0, 0.40824829046386296, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999998, 0.4999999999999999, 0.0, 0.0, 0.7071067811865475, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [94]:
# Collaborative 

#print(len(restaurant_data_clean))

user_swiped = [[-1 , 0, 1, 1], [1 ,0, 0, 1]]
user_swiped = rng.integers(low = 0, high = 2, size = (3, len(restaurant_data_clean)))
print(user_swiped.shape)
R = user_swiped
print(R.shape)
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)

(3, 4204)
(3, 4204)


In [95]:
print(list(user_swiped[0]))

[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 

In [96]:
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Global bias:")
print(mf.b)
print()
print("User bias:")
print(mf.b_u)
print()
print("Item bias:")
print(mf.b_i)

Iteration: 10 ; error = 0.0628
Iteration: 20 ; error = 0.0181

P x Q:
[[1.00003228 0.99996936 0.99993057 ... 1.00004011 1.00007087 1.00002157]
 [0.99996107 1.00001429 1.00001944 ... 0.99995246 1.00004392 0.99997843]
 [0.99994752 1.00001446 1.00005387 ... 1.00000567 1.00002126 0.9999716 ]]

Global bias:
1.0

User bias:
[4.45873637e-06 1.49164222e-05 9.97358782e-06]

Item bias:
[-2.25032506e-04  8.21908809e-05  3.91434187e-04 ...  2.21870345e-04
 -1.47940622e-04 -1.21932261e-04]


In [120]:
"""
user_user: [user_1: [item_1, item_2, item_3, item_4], user_2: [item1, item_2, item_3, item_4]
"""
user_user = mf.full_matrix()
sorted(user_user[0], reverse = False)

[0.9956000366695834,
 0.9957163755460771,
 0.9967940200817765,
 0.9972900828603926,
 0.9976273361017176,
 0.9976471708143606,
 0.9977256171397377,
 0.9977524051678378,
 0.9978997555689787,
 0.9979737466352664,
 0.9981801135434217,
 0.9983557880435622,
 0.9984418251254032,
 0.9986435050547791,
 0.9987495850955729,
 0.998808708546085,
 0.998889105891633,
 0.9989725770979904,
 0.9990546868735831,
 0.9990552699617931,
 0.9990596163992771,
 0.9992232442569784,
 0.9992308099369278,
 0.9992621573204911,
 0.9992632941373365,
 0.999350684507454,
 0.999373779296756,
 0.9993985630970685,
 0.9994045948683518,
 0.9994211321812521,
 0.9995054475531021,
 0.9995070651227448,
 0.9995104576846051,
 0.9995149676116666,
 0.999517590210732,
 0.9995345418846957,
 0.9995448793289576,
 0.9995509449565233,
 0.9995688165878581,
 0.9995792656223697,
 0.9995887521577852,
 0.9995911264297327,
 0.9996007594466445,
 0.9996020147050863,
 0.9996081525983792,
 0.9996122256164613,
 0.9996175602402065,
 0.999618508104073

In [98]:
'''
item_item: [item1: [item_1, item_2, item_3, item_4], item_2: [item_1, item_2, item_3, item_4] ...]
'''
item_item_matrix[0]

[0.9999999999999998,
 0.0,
 0.4999999999999999,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.4999999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8164965809277259,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4999999999999999,
 0.0,
 0.0,
 0.4999999999999999,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4999999999999999,
 0.4999999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,


In [99]:
'''
item_user: [user_1: [item_1, item_2, item_3, item_4], user_2:[ item_1, item_2, item_3, item_4]]
'''
item_user[0]

array([0.5 , 0.5 , 0.25, ..., 0.25, 0.25, 0.25])

In [100]:
for i in range(0, len(item_user[0])):
    if(item_user[0][i] == 0.5):
        print(item_user[0][i], user_user[0][i])

0.5 0.731064925168179
0.5 0.7310525544039699
0.5 0.7310553836480493
0.5 0.7310661537955735
0.5 0.7310553363859676
0.5 0.7310798139570794
0.5 0.7310592501782772
0.5 0.7310598964357652
0.5 0.731065570227759
0.5 0.7310653905818479
0.5 0.7310671778840061
0.5 0.7310105709713923
0.5 0.7310455814043548
0.5 0.7310011735183889
0.5 0.7310449131584253
0.5 0.7310340356932667
0.5 0.7310781516968534
0.5 0.7310807182861658
0.5 0.7310263371691433
0.5 0.731021312042304
0.5 0.7310550313006517
0.5 0.7310919243510093
0.5 0.731058480895697
0.5 0.7310572907856772
0.5 0.7310583391839954
0.5 0.7310565288810869
0.5 0.7310585653188767
0.5 0.731068501347265
0.5 0.7310436616679115
0.5 0.7309978803449745
0.5 0.7310822335638503
0.5 0.7310727459317089
0.5 0.7310602701308381
0.5 0.7310658644880047
0.5 0.7310583197569405
0.5 0.7310718467161856
0.5 0.7310595087915669
0.5 0.7310418602181091
0.5 0.7310764697491877
0.5 0.7310492384503151
0.5 0.7310584316932737
0.5 0.7310509805826233
0.5 0.7310501106657004
0.5 0.7310662044

In [101]:
def get_collab_content(item_user_arr, user_user_arr):
    collab_content = np.array(item_user_arr) + np.array(user_user_arr)
    return collab_content

In [102]:
collab_content = get_collab_content(item_user, user_user).tolist()
collab_content[0]
sorted(collab_content[0], reverse= True)

[1.731078384609196,
 1.4810986877183026,
 1.481094696885199,
 1.4810908945187462,
 1.481083666709161,
 1.4810823921343794,
 1.4810823458057483,
 1.481080429748654,
 1.4810778279270194,
 1.4810770542125262,
 1.4810722031812396,
 1.4810715127448066,
 1.481069686662091,
 1.4810691006684333,
 1.4810686210740776,
 1.481068130759334,
 1.4810666240007686,
 1.4810662889532118,
 1.4810661715390823,
 1.481064781190947,
 1.481063967461902,
 1.4810627750276923,
 1.4810615822981057,
 1.4810608056772243,
 1.481060509602795,
 1.4810586859008121,
 1.4810586651420738,
 1.4810584277550916,
 1.481058270105891,
 1.4810580216491416,
 1.4810577497168733,
 1.4810577224765442,
 1.4810568523892402,
 1.481056000599643,
 1.4810559812293849,
 1.4810556950435712,
 1.4810554184618194,
 1.4810549021042112,
 1.481054210558083,
 1.4810541824760488,
 1.4810539971974181,
 1.4810535738473383,
 1.4810533066307854,
 1.4810525196511555,
 1.4810523731445135,
 1.4810512857871618,
 1.4810507967219833,
 1.4810506494484224,
 1.4

In [103]:
def get_recommendation(user_index, item_item_matrix, collab_content):
    user_arr = collab_content[user_index]
    recommendation = user_arr.index(max(user_arr))
    print(recommendation)
    # Recommend the top 5 which the user has not swiped on
    n_recommendations = sorted(range(len(item_item_matrix[recommendation])), key=lambda i: item_item_matrix[recommendation][i], reverse = True)
    for i in n_recommendations:
        if(user_swiped[user_index][i] == 1):
            n_recommendations.remove(i)
    #print(item_item_matrix[recommendation])
    #print(sorted(item_item_matrix[recommendation], reverse=True))
    print(n_recommendation)
    return n_recommendation

array([[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1]])

In [114]:
user_np = np.load('../user.npy')
# read json file    
import json
with open('../user.json') as f:
    user_swiped_on = json.load(f)
user_swiped_on

{'default': [0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,

In [119]:
print(user_swiped_on['default'])

[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 

In [104]:
recco = get_recommendation(0, item_item_matrix, collab_content)

878
[878, 2185, 2187, 4142, 1671]


In [105]:
print(user[0])

[1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0]


In [106]:
sample_item = item[1684]
sample_user = user[0]
print(sample_item)
print(sample_user)
#print(sigmoid(dot(sample_user, sample_item.T)))

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]
[1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0]


In [107]:
print(sorted(item_item_matrix[1684], reverse = True))

[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259,

In [110]:
# User Cuisine
#print(user[0])
# User Swipe
# Restaurant Cuisne
default_recc = recco[0]
for recc in recco:
    print("restaurant name:", restaurant_name_list[recc])
    print("item-item:", item_item_matrix[default_recc][recc])
    print("user swiped:", user_swiped[0][recc], "item-user-similarity:", item_user[0][recc] ,"user-user-similarity:", user_user[0][recc])
    print("collab_content", collab_content[0][recc])
    for i in range(0, 22):
        print(restaurant_data_clean.columns[i+1],"u:", user[0][i], "r:", restaurant_data_clean.iloc[recc, i+1])

restaurant name: dbespoke
item-item: 1.0
user swiped: 0 item-user-similarity: 1.0 user-user-similarity: 0.7310783846091959
collab_content 1.731078384609196
japanese u: 1 r: 1
chinese u: 0 r: 0
bar u: 1 r: 1
western u: 0 r: 0
indian u: 0 r: 0
mediterranean u: 1 r: 0
mexican u: 1 r: 0
italian u: 1 r: 1
local u: 1 r: 0
malaysian u: 0 r: 0
hipster u: 1 r: 1
veg u: 0 r: 0
fast_food u: 1 r: 0
thai u: 1 r: 0
korean u: 0 r: 0
halal u: 0 r: 0
dessert u: 0 r: 0
seafood u: 1 r: 0
health u: 1 r: 0
french u: 1 r: 0
vietnamese u: 1 r: 0
michellin u: 0 r: 0
restaurant name: ma maison
item-item: 0.8660254037844387
user swiped: 0 item-user-similarity: 0.75 user-user-similarity: 0.7310908945187462
collab_content 1.4810908945187462
japanese u: 1 r: 1
chinese u: 0 r: 0
bar u: 1 r: 0
western u: 0 r: 0
indian u: 0 r: 0
mediterranean u: 1 r: 0
mexican u: 1 r: 0
italian u: 1 r: 1
local u: 1 r: 0
malaysian u: 0 r: 0
hipster u: 1 r: 1
veg u: 0 r: 0
fast_food u: 1 r: 0
thai u: 1 r: 0
korean u: 0 r: 0
halal u: 0 

## Miscallaneous from here on

In [255]:
df_grab = pd.read_csv("complete_grab.csv")

In [43]:
import json
from ast import literal_eval

df_grab

df_tran = pd.DataFrame({})
ind = 0
for cuisine_arr in df_grab['cuisine']:
    i = 1
    dict_foodie = {}
    
    if(type(cuisine_arr) != float):
        cuisine_arr = literal_eval(cuisine_arr)
        for cuisine in cuisine_arr:
            string_iter = 'cuisine'+str(i)
            dict_foodie[string_iter] = 1
            i=i+1
    df_foodie = pd.DataFrame.from_records(dict_foodie, index = [ind])
    df_foodie = pd.concat([df_grab, df_foodie])
    ind+=1

In [21]:
df_foodie

,cuisine1,cuisine10,cuisine11,cuisine12,cuisine13,cuisine14,cuisine15,cuisine16,cuisine17,cuisine18,...,cuisine42,cuisine43,cuisine44,cuisine45,cuisine46,cuisine5,cuisine6,cuisine7,cuisine8,cuisine9
9661,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
